***

* [Outline](../0_Introduction/0_introduction.ipynb)
* [Glossary](../0_Introduction/1_glossary.ipynb)
* [7. Observing Systems](0_introduction.ipynb)  
    * Previous: [7.3 Analogue Electronics](7_3_analogue.ipynb)
    * Next: [7.5 The Primary Beam](7_5_primary_beam.ipynb)

***

Import standard modules:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import HTML 
HTML('../style/course.css') #apply general CSS

In [ ]:
HTML('../style/code_toggle.html')

***

**TODO**

* introduce correlator, describe FX
* define cross- and auto-correlation
* relate correlator to vCZ
* digital FX correlator block diagram
* FX correlator architecture block diagram
* component: coarse delay
* component: FFT, FIR, PFB
* component: fine delay, fringe-stopping
* component: gain correction, quantization
* component: cornerturn
* component: correlation, Vacc
* Output to measurement set with metadata (uvw, time, observation information, frequency, flags)

## 7.4 Digital Electronics

At the heart of an interferometric array is a digital *correlator*, this channelizes and correlates every pair of antenna signals to compute the baseline correlations. There are two types of correlation, the *cross-correlation* which is each antenna correlated with all other antennas in the array. For an array of $N_{\textrm{antennas}}$ antennas there are $\frac{N_{\textrm{antennas}} (N_{\textrm{antennas}}-1)}{2}$ cross-correlations, the factor of a half is there to account for double counting as the correlation of antenne $i$ with antenna $j$ is the same (up to a complex conjugation) of antenna $j$ with antenna $i$. The other type of correlation is the *auto-correlation* which the correlation of the antenna with itself. There are $N_{\textrm{antennas}}$ auto-correlations. Generally in interferometry, the auto-correlations are ignored as they are dominated by system noise. The total number of correlations performed in the correlator for an array with $N_{\textrm{antennas}}$ antennas is

$$
N_{\textrm{corr}} = N_{\textrm{auto corr}} + N_{\textrm{cross corr}} = N_{\textrm{antennas}} + \frac{N_{\textrm{antennas}} (N_{\textrm{antennas}}-1)}{2}
$$

A typical digital back-end for a radio astronomy dish is a spectrometer, beamformer, pulsar timing engine, base-band recorder, or a correlator. These instruments do fairly simple operations, but these operations need to be performed very quickly. A *spectrometer* transform a time-domain voltage signal into a frequency-domain spectrum using a *fast Fourier Transform (FFT)*, a power converter, and an accumulator. A *beamformer* adds together multiple antenna signals into one combined signal. The simplest digital instrument is the *base-band recorder* which simply records an ADC output signal to disk, as in the case of many VLBI systems. *Correlator* are the important instrument for interferometry. Two signals are correlated together by use of an FFT, multiplier, and accumulator. We will focus on the correlator in this discussion.

### 7.4.1 Nyquist Sampling and Aliasing

Digitization samples a continuous signal at regular intervals, this introduces a frequency ambiguity due to the $f_s$ sampling rate. With a sampling rate of $f_s$, the maximum frequency which can be resolved (the Nyquist frequency) is $f_s/2$. Frequencies above the Nyquist frequency are said to 'alias'. If we are only sampling at a rate of $f_s$ then any sine wave oscillating at a higher frequency can go through many cycles between sampling points. There exists another sine wave, going through fewer cycles at below the Nyquist frequency which will match in amplitude at the same sampling points. Thus these two sine wave can not be differentiated, in fact there are not just two but an infinite set of such waves due to the periodic nature of the sine wave. Knowing this, if we would like to Nyquist sample a 100 MHz band, then we need an ADC which samples at 200 MHz, with appropriate bandpass filters. This is a difficult topic and there are many books which will provide a thorough explanation of the topic, a good starting point is Richard Lyons' <cite data-cite='Lyons:2004:UDS:993484'>Understanding Digital Signal Processing</cite> [CITE](https://books.google.co.za/books?id=UBU7Y2tpwWUC).

### 7.4.2 Correlators

The convolution theorem has been discussed in previous chapters but as a reminder, the theorem states that for two signals $f$ and $g$

$$
\mathcal{F}\{ f \circ g \} = \mathcal{F}\{f\} \cdot \mathcal{F}\{g\}
$$

where $\mathcal{F}\{ \cdot \}$ is the Fourier transform of the signal and $\ast$ is the convolution operator. The convolution of two signals $h(z)$ is

$$
h(z) = f \circ g = \int f(x) g(z-x)dx
$$

Correlation and convolution are very similar, the difference lies in the details of using complex signals and using the conjugate of one of the signals. The importance in these equations is that by multiplying the Fourier transform of two signals we get the Fourier transform of the two functions convolved. A convolution is a computationally expensive task, where as two FFTs and a multiplication is fairly simple. By convolving sky signals from two dishes we produce the *interferometer visibility*.

In a practical way how are the visibilities computed with a correlator? There are two ways to go about it, the FX and XF architectures, but both ways produce the same result in the end. In the FX architecture a Fourier Transform is applied to a window of samples from two signals, this is the F component. Then the two Fourier transformed signals are multiplied together (one of the signals is conjugated to make this a correlation instead of a convolution), and the resulting signal is then accumulated, this is the X component. As you might guess, the XF architecture does the two operations in a reverse order. There are a few reasons to choose one architectures over the other but, suffice to say the FX is generally favoured for computational reasons. The correlation of all antenna pairs is computationally intensive operation and is commonly implemented on field programmable gate arrays (FPGAs) or graphics processing units (GPUs).

### 7.4.3 FFTs and Poly-Phase Filter Banks

The bandpass of the analogue system and digitization process select out a range of frequencies to be correlated between antennas. The full band from each antenna can be correlated against another antenna but this would only give information about the entire band. If there is strong narrowband RFI then any correlation will be dominated by that signal, and the sky signal will be lost. Or, if we are only interested in a weak narrow signal, such as a spectral line, that line structure will be hidden in the band correlation. In order to gain access to subsets of the frequency range, known as *subbands* or *channels*, a Fourier Transform must be employed. In this context a Fourier Transform will transform a wideband time-domain signal into a set of frequency-domain signals (for a given 'window' of samples). As we are working on discrete samples, regularly sampled in time we will use the *Discrete Fourier Transform (DFT)* which is defined as

$$
\label{eq:fourier}
X(k) = \sum^{N-1}_{n=0} x(n) e^{\frac{-i 2 \pi}{N} n k}
$$

where $N$ is the number of samples in the data window. Using this form the naive computational cost of this transform is $\mathcal{O}(N^2)$, which can grow to be very computationally expensive for relatively small sizes of $N$. The solution to this computational problem is the *Fast Fourier Transform (FFT)* which is a class of algorithms with use the regularity of the sampling to approach a run time of $\mathcal{O}(N \log N)$. The Cooley-Tukey Radix-2 algorithm is the classic example of a FFT algorithm. An important note to make right here, you will often see FFTs of size $2^d$, i.e. 256, 512, 1024, etc., being used, this is because when a size of $2^d$ is used the most efficient FFT can be used. Other FFT sizes, even sizes $<2^d$ will take longer to compute.
So, the point I am trying to make is *any time you are doing an FFT try to pick a size which is of the form $2^d$*.

Back to our discretely-sampled time-domain signal, depending on how many sampled we choose to include in our window for applying the FFT affects the subband resolution. For $N$ real-valued time-domain signals $N/2$ complex frequency subbands are created. The apparent factor of 2 'loss' in information due to the FFT is because the output of the FFT is complex and the input is real, so there are effectively 2 values per frequency subband. The more samples included in the window, the higher the resolution of subbands output. In the extreme case of an infinite window the DFT approaches the continuous Fourier Transform output. On the other side, a window of 1 sample will produce the same output as input. It is often useful to think of the FFT generating a series of narrow-band time domain signals, this is called a *filterbank* is DSP.

I have been using the term windows for blocks of samples which an FFT is applied to. If you take a continuous signal and select out a block of samples then you have altered the signal, you have effectively applied a *top-hat function* (rectangle) to the signal. This might not seem like an issue because none of the samples have been modified, but the continuity of the signal has been altered. When an FFT is applied to the window, from the convolution theorem, we are convolving the Fourier transform of the data with the Fourier transform of a top-hat function. The Fourier transform of the top-hat function is known as a *sinc function* which is defined as $\textrm{sinc}(x) = \frac{\sin x}{x}$. The top-hat function is known as a *windowing function*, and it turns out to be a generally poor one. The primary issue with the top-hat function is that is that there are high 'sidelobes' with a 'slow' roll-off. Many other windowing functions have been developed: Hamming, Hann, Blackman, Gaussian, etc. These windowing functions have a trad off in the subband width, sidelobe levels, sidelobe roll-off, and sharpness. A simple example as to why you would want to apply a windowing function to samples is that if there is strong RFI in one subband, and the windowing function has high sidelobes (like a top-hat function), then power will *leak* from the RFI subband to clean subband, corrupting it.

A more advanced form of windowing is often applied before the FFT in what is known as a *finite impulse response* (FIR) filter which uses overlapping windowed samples to have a larger winding function. The FIR together with the FFT is known as a *poly-phase filterbank (PFB)*, all modern digital back-ends use this instead of the simple FFT.

### 7.4.4 Quantized Correlation

After the bandpass signal has been channelized into subbands then the subbands are ready to be multiplied and accumulated. That is, for each antenna pair $A_i, A_j$, subband $k$ from antenna $A_i$ is multiplied with (the complex conjugate of) subband $k$ of Antenna $A_j$. Only subbands with the same frequency are correlated. For an array of $N_{\textrm{antennas}}$ antenna elements, the number of baseline correlations is

$$
\label{eq:baselines}
N_{\textrm{corr}} = N_{\textrm{auto corr}} + N_{\textrm{cross corr}} = N_{\textrm{antennas}} + \frac{N_{\textrm{antennas}} (N_{\textrm{antennas}}-1)}{2}
$$

where $N_{\textrm{auto corr}}$ is the number of antenna self-correlations, or \emph{auto-correlations} and $N_{\textrm{cross corr}}$ is the number of antenna pair \emph{cross-correlations}. In most cases when using an interferometric array, the auto-correlations are ignored due to issues of antenna self-noise, but that is a discussion for another time.

Before the multiplication and accumulation is performed that is a stage known as quantization. This stage reduces down the number of bits used to represent the signal in each subband, often down to 1, 2, or 4 bits per value. Quantization is performed to save in computational cost, and it turns out to have minimal impact on a a sky signal. For example, quantizing a subband signal down to 4 bits for the real component and 4 bit for the imaginary component, there is a *quantization efficiency* of 0.989. That is, as opposed to an unquantized signal, there is only a 1.1% loss in correlation. Going down to 1 bit (2 levels) the quantization efficiency is 0.637, not too bad for a single bit.

After quantization the multiplication are performed, and accumulated to increase the signal to noise and reduce the output data rate. The final visibilities are collected into data cubes with dimensions of time, frequency, and polarization for each baseline.

This digital back-end overview has skipped some additional features included in these systems, such as timing keeping, fine and coarse delay correction, equalization, and fringe-stopping. All these features are important in the practical design on an interferometric array, and are worth reading up on.

***

Next: [7.5 The Primary Beam](7_5_primary_beam.ipynb)

<div class=warn><b>Future Additions:</b></div>